In [1]:
import numpy as np

## Calcularea functiei de cost pentru regresia logistica multinomiala

Formula costului pentru $\theta$ matrice de k linii si n coloane este un scalar cu expresia data in curs:

$$J(\theta) = - \frac{1}{m} \left[ 
    \sum_{j=1}^{m} \sum_{l=1}^{k} I(y^{(j)} = l) \cdot
    log \frac{
        exp( \theta_l^t x^{(j)} )
    }{
        \sum_{h=1}^{k} exp( \theta_h^t x^{(j)} )
    } \right]$$
    
$x$ este matrice de m x n si 

$\theta$ matrice de k x n.

Daca ne referim la $x^{(j)}$ vorbim de linia j din matricea $x$, care linie are n elemente, iar daca ne referim la $\theta_l$ discutia este despre linia l din matricea $\theta$, care linie are, iarasi, tot n elemente.

Consideram calculul termenului logaritm din relatia anterioara dar folosind relatiile matriciale. Termenul $\theta_l^t x^{(j)}$, care este un scalar, poate fi calculat matricial cu produsul matricial $x \cdot \theta^t$, care va avea dimensiunea de m x k.

In particular, se observa ca dupa exponentierea matricei rezultat, este necesar sa calculam suma pe fiecare coloana in parte (sum cu axis = 1). Pe urma impartim fiecare element din matricea rezultat cu suma de pe coloana sa astfel calculata. Este o simpla impartire element-cu-element cu vectorul de m elemente sume calculat anterior (se face broadcast de la forma m x 1 la forma m x k).

Mai departe, pentru ca matricea logaritm de m x k sa fie compatibila cu I(.) (functia indicator), trebuie sa scriem functia indicator ca o matrice m x k, in care fiecare linie este un "one-hot encoding" pentru eticheta pattern-ului $x^{(j)}$. 

Astfel extinsa, functia indicator de m x k se poate inmulti element-cu-elemnet cu matricea logaritm. Rezultatul este tot o matrice m x k. Costul scalar J(.) se obtine prin sumarea pe linii (np.sum, axis = 1) si pe urma prin sumarea pe coloane (a ramas o singura linie), sau, mai rapid, prin folosirea functiei np.average pe coloane.

## Calcularea gradientului pentru regresia logistica multinomiala

Pornim de la expresia algebirca a gradientului in raport cu directiile date de vectorul $\theta_l$ (care are n componente, unde l parcurge 1..k, cum am vazut mai sus):

$$\nabla_{\theta_l}(\theta) = - \frac{1}{m} \sum_{j=1}^{m} \left[
    x^{(j)} \left(
        I(y^{(j)} = l) - \frac{
            exp( \theta_l^t x^{(j)} )
        }{
            \sum_{h=1}^{k} exp( \theta_h^t x^{(j)} )
        }
    \right)
\right]$$

Argumentul de sub logaritmul din prima relatie este functia softmax(.), pe care am expandat-o aici. Dimensiunea matricii lui softmax() este m x k.

Pentru a o face compatibila cu functia indicator, I(.) este scrisa, ca mai sus, cu forma m x k, unde fiecare linie este un one-hot encoding. Mai departe, ne referim la diferenta I(.) - softmax(.) ca la matricea "ip", de forma m x k.

Acum urmeaza partea interesanta. Fiecare linie $x^{(j)}$ de n elemente se inmulteste cu fiecare din elementele liniei j din matricea ip. Fie matricile x si ip astfel:

$$ 
x = \left[ \begin{array}{cccc}
x_1^{(1)} & x_2^{(1)} & \dots & x_n^{(1)} \\
x_1^{(2)} & x_2^{(2)} & \dots & x_n^{(2)} \\
\dots & \dots & \dots & \dots \\
x_1^{(m)} & x_2^{(m)} & \dots & x_n^{(m)} \\
\end{array} \right]
\qquad 
ip = \left[ \begin{array}{cccc}
ip_1^{(1)} & ip_2^{(1)} & \dots & ip_k^{(1)} \\
ip_1^{(2)} & ip_2^{(2)} & \dots & ip_k^{(2)} \\
\dots & \dots & \dots & \dots \\
ip_1^{(m)} & ip_2^{(m)} & \dots & ip_k^{(m)} \\
\end{array} \right]
$$

Fiecare din elementele liniei j din matricea x se inmulteste asadar cu fiecare element de pe linia corespondenta j din matricea ip. Vizualizati urmatorul lucru: matricea x se "trasforma" intr-un tensor tridimensional, in care fiecare element capata o "adancime" de k elemente. Mai departe, pe acest tensor 3D se face o suma intre toate cele m linii (prima dimensiune), si rezulta o matrice n x k.


O varianta mai simpla, pentru ca tot se face sumare pe linii, este sa inmultim matricial x si ip. Iata mai jos, pe un exemplu, cele doua strategii:

In [19]:
# alegem dimensiunile sa fie diferite
m = 3 
n = 4
k = 2

# matricea x este de forma (m x n)
x = np.array([[1, 2, 3, 4], [2, 1, -1, 2], [3, 1, 4, -2]])
assert(x.shape == (m, n))

# matricea ip este de forma (m x k)
ip = np.array([[2, 1], [3, 4], [5, 6]])
assert(ip.shape == (m, k))

### Strategia folosind tensori 3D

Copiem cele doua matrici:

In [21]:
x_ = x.copy()
ip_ = ip.copy()

Facem extindere la 3 dimensiuni:

In [23]:
x_ = np.reshape(x_, (m, 1, n))
ip_ = np.reshape(ip_, (m, k, 1))

Inmultim element cu element cele doua matrici. Deoarece dimensiunile nu coincid, se va face broadcasting pe dimensiunile cu valoare 1.

Consideram 3 dimensiuni, "linii", "coloane" si "adancimi".

Pentru x, deoarece este o singura coloana, planul de linii si adancimi se replica pentru k coloane. Pentru ip, deoarece exista o singura adancime, planul de linii si coloane se replica pentru n adancimi. (incercati sa vizualizati acest lucru).

In [24]:
delta_ = x_ * ip_

Mai departe, sumam intre linii (axa 0). Ca urmare, o sa ramana doar un plan orizontal, cu o singura linie de coloane si adancimi. Acesta este theta.

In [25]:
delta_ = np.sum(delta_, axis=0)
delta_.shape

(2, 4)

### Strategia folosind inmultirea de matrici

Mai simplu, inglobam si suma dupa m si folosim produsul matricial:

In [30]:
delta = np.dot(ip.T, x)
delta.shape

(2, 4)

Comparam rezultatele obtinut din ambele strategii:

In [29]:
assert(np.all(delta == delta_))